<a href="https://colab.research.google.com/github/albey-code/hippoabstraction/blob/main/GLM_FirstLevelModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_path = '/content/drive/MyDrive/Mona_Garvert_2017_fMRI_Data'

In [3]:
# Define paths to fMRI .nii.gz, events.tsv, and motion .tsv files

bold_paths = [
    f"{base_path}/trimmed/sub-04_run-1_bold_trimmed.nii.gz",
    f"{base_path}/trimmed/sub-04_run-2_bold_trimmed.nii.gz",
    f"{base_path}/trimmed/sub-04_run-3_bold_trimmed.nii.gz"
]

confound_paths = [
    f"{base_path}/sub-04_run-1_motion_physio.tsv",
    f"{base_path}/sub-04_run-2_motion_physio.tsv",
    f"{base_path}/sub-04_run-3_motion_physio.tsv"
]

event_paths = [
    f"{base_path}/sub04_run1_events.tsv",
    f"{base_path}/sub04_run2_events.tsv",
    f"{base_path}/sub04_run3_events.tsv"
]

In [5]:
#! pip install nilearn
import pandas as pd
import nibabel as nib

In [6]:
from nilearn.glm.first_level import FirstLevelModel

# GLM based on Garvert et al.'s (2017) Matlab script

first_level_model = FirstLevelModel(
    t_r=3.01,
    slice_time_ref=15/43,     # Equivalent to SPM’s fmri_t0 = 15 with 43 slices
    hrf_model='spm',
    high_pass=1/128,          # Nilearn uses Hz, so 1/128 s #See line 147 of mydesign_1007.m
    noise_model='ar1',        # See fmri_spec.cvi = 'AR(1)'
    standardize=False,        # No mention of scaling, so I disabled it
    signal_scaling=0,         # To match SPM’s default (no scaling)
    minimize_memory=False
)

# Run 1

In [7]:
# Run 1 Path Definition
bold_path_run1 = f"{base_path}/trimmed/sub-04_run-1_bold_trimmed.nii.gz"
confound_path_run1 = f"{base_path}/sub-04_run-1_motion_physio.tsv"
event_path_run1 = f"{base_path}/sub04_run1_events.tsv"

In [8]:
# Load event and confound data
events_run1 = pd.read_csv(event_path_run1, sep='\t')
confounds_run1 = pd.read_csv(confound_path_run1, sep='\t', header=None) #cave, the header in GLM_motion_regressors.ipynb is set to False, so the same applies here!!

In [40]:
# Check the number of volumes in BOLD image, the number of rows in confounds and the number of rows in confounds after check/trim -> they should all be 442 for run 1 subject 04

Number of volumes in BOLD image: 442
Number of rows in confounds: 442
Number of rows in confounds after check/trim: 442


In [9]:
# Reload the file to check whether header exists #Just a sanity check to ensure no important info. was accidentally omitted by setting header=None
#df = pd.read_csv(confound_path_run1, sep='\t', header=None)
#print(df.head())

Fit the First Level GLM (see above) for Run 1

In [10]:
# Fit First Level GLM for Run 1
first_level_model = first_level_model.fit(bold_path_run1, events=events_run1, confounds=confounds_run1)

/usr/local/lib/python3.11/dist-packages/nilearn/glm/first_level/design_matrix.py:416: UserWarning: The following conditions contain events with null duration:
- 'button_press'
- 'object_10'
- 'object_2'
- 'object_4'
- 'object_6'
- 'object_7'
- 'object_8'
- 'object_9'

  matrix, names = _convolve_regressors(


Inspect the design matrix X from the FirstLevelModel GLM above

In [11]:
import matplotlib.pyplot as plt

In [14]:
from nilearn.plotting import plot_design_matrix

# Plot + Inspect the design matrix
#dm_run1 = first_level_model.design_matrices_[0]
#plot_design_matrix(dm_run1)
#plt.show()

In [15]:
design_matrix = first_level_model.design_matrices_[0]
print(design_matrix.columns.tolist())

['button_press', 'object_10', 'object_2', 'object_4', 'object_6', 'object_7', 'object_8', 'object_9', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'constant']


Compute beta estimate maps for all 7 object regressors, 1 button press and 1 patch regressor

In [20]:
from nilearn.glm.contrasts import compute_contrast #Cave! This is not for acc. computing contrasts between objects, I am taking the 'raw' effect sizes

# Repeat for all 7 object regressors
object_2_beta_map = first_level_model.compute_contrast('object_2', output_type='effect_size')
object_4_beta_map = first_level_model.compute_contrast('object_4', output_type='effect_size')
object_6_beta_map = first_level_model.compute_contrast('object_6', output_type='effect_size')
object_7_beta_map = first_level_model.compute_contrast('object_7', output_type='effect_size')
object_8_beta_map = first_level_model.compute_contrast('object_8', output_type='effect_size')
object_9_beta_map = first_level_model.compute_contrast('object_9', output_type='effect_size')
object_10_beta_map = first_level_model.compute_contrast('object_10', output_type='effect_size')

# Repeat for button press regressor
button_press_beta_map = first_level_model.compute_contrast('button_press', output_type='effect_size')

# Repeat for 'patch' regressor
#patch_beta_map = first_level_model.compute_contrast('patch', output_type='effect_size') #potentially I forgot this regressor, need to check!

Save beta maps locally for later downstream analyses

In [22]:
import zipfile
import nibabel as nib
from google.colab import files

In [23]:
# Dictionary of beta maps I already created
beta_maps = {
    'object_2': object_2_beta_map,
    'object_4': object_4_beta_map,
    'object_6': object_6_beta_map,
    'object_7': object_7_beta_map,
    'object_8': object_8_beta_map,
    'object_9': object_9_beta_map,
    'object_10': object_10_beta_map,
    'button_press': button_press_beta_map
}

# Save all beta maps
for label, beta_map in beta_maps.items():
    filename = f"sub-04_run1_{label}_beta.nii.gz"
    nib.save(beta_map, filename)

# Zip them
with zipfile.ZipFile("sub-04_run1_betas.zip", "w") as zipf:
    for label in beta_maps.keys():
        zipf.write(f"sub-04_run1_{label}_beta.nii.gz")

# Download the zip
files.download("sub-04_run1_betas.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run 2

In [24]:
# Run 2 Path Definition
bold_path_run2 = f"{base_path}/trimmed/sub-04_run-2_bold_trimmed.nii.gz"
confound_path_run2 = f"{base_path}/sub-04_run-2_motion_physio.tsv"
event_path_run2 = f"{base_path}/sub04_run2_events.tsv"

In [26]:
# Load event and confound data
events_run2 = pd.read_csv(event_path_run2, sep='\t')
confounds_run2 = pd.read_csv(confound_path_run2, sep='\t', header=None) #cave, the header in GLM_motion_regressors.ipynb is set to False, so the same applies here!!

Fit the First Level GLM (see above) for Run 2

In [28]:
# Fit First Level GLM for Run 2
first_level_model = first_level_model.fit(bold_path_run2, events=events_run2, confounds=confounds_run2)

/usr/local/lib/python3.11/dist-packages/nilearn/glm/first_level/design_matrix.py:416: UserWarning: The following conditions contain events with null duration:
- 'button_press'
- 'object_10'
- 'object_2'
- 'object_4'
- 'object_6'
- 'object_7'
- 'object_8'
- 'object_9'

  matrix, names = _convolve_regressors(


Inspect the design matrix X from the FirstLevelModel GLM above

In [29]:
design_matrix = first_level_model.design_matrices_[0]
print(design_matrix.columns.tolist())

['button_press', 'object_10', 'object_2', 'object_4', 'object_6', 'object_7', 'object_8', 'object_9', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'constant']


Compute beta estimate maps for all 7 object regressors, 1 button press and 1 patch regressor

In [30]:
from nilearn.glm.contrasts import compute_contrast #Cave! This is not for acc. computing contrasts between objects, I am taking the 'raw' effect sizes

# Repeat for all 7 object regressors
object_2_beta_map = first_level_model.compute_contrast('object_2', output_type='effect_size')
object_4_beta_map = first_level_model.compute_contrast('object_4', output_type='effect_size')
object_6_beta_map = first_level_model.compute_contrast('object_6', output_type='effect_size')
object_7_beta_map = first_level_model.compute_contrast('object_7', output_type='effect_size')
object_8_beta_map = first_level_model.compute_contrast('object_8', output_type='effect_size')
object_9_beta_map = first_level_model.compute_contrast('object_9', output_type='effect_size')
object_10_beta_map = first_level_model.compute_contrast('object_10', output_type='effect_size')

# Repeat for button press regressor
button_press_beta_map = first_level_model.compute_contrast('button_press', output_type='effect_size')

# Repeat for 'patch' regressor
#patch_beta_map = first_level_model.compute_contrast('patch', output_type='effect_size') #potentially I forgot this regressor, need to check!

Save beta maps locally for later downstream analyses

In [31]:
import zipfile
import nibabel as nib
from google.colab import files

In [32]:
# Dictionary of beta maps I already created
beta_maps = {
    'object_2': object_2_beta_map,
    'object_4': object_4_beta_map,
    'object_6': object_6_beta_map,
    'object_7': object_7_beta_map,
    'object_8': object_8_beta_map,
    'object_9': object_9_beta_map,
    'object_10': object_10_beta_map,
    'button_press': button_press_beta_map
}

# Save all beta maps
for label, beta_map in beta_maps.items():
    filename = f"sub-04_run2_{label}_beta.nii.gz"
    nib.save(beta_map, filename)

# Zip them
with zipfile.ZipFile("sub-04_run2_betas.zip", "w") as zipf:
    for label in beta_maps.keys():
        zipf.write(f"sub-04_run2_{label}_beta.nii.gz")

# Download the zip
files.download("sub-04_run2_betas.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run 3

In [25]:
# Run 3 Path Definition
bold_path_run3 = f"{base_path}/trimmed/sub-04_run-3_bold_trimmed.nii.gz"
confound_path_run3 = f"{base_path}/sub-04_run-3_motion_physio.tsv"
event_path_run3 = f"{base_path}/sub04_run3_events.tsv"

In [27]:
# Load event and confound data
events_run3 = pd.read_csv(event_path_run3, sep='\t')
confounds_run3 = pd.read_csv(confound_path_run3, sep='\t', header=None) #cave, the header in GLM_motion_regressors.ipynb is set to False, so the same applies here!!

Fit the First Level GLM (see above) for Run 3

In [33]:
# Fit First Level GLM for Run 3
first_level_model = first_level_model.fit(bold_path_run3, events=events_run3, confounds=confounds_run3)

/usr/local/lib/python3.11/dist-packages/nilearn/glm/first_level/design_matrix.py:416: UserWarning: The following conditions contain events with null duration:
- 'button_press'
- 'object_10'
- 'object_2'
- 'object_4'
- 'object_6'
- 'object_7'
- 'object_8'
- 'object_9'

  matrix, names = _convolve_regressors(


Inspect the design matrix X from the FirstLevelModel GLM above

In [34]:
design_matrix = first_level_model.design_matrices_[0]
print(design_matrix.columns.tolist())

['button_press', 'object_10', 'object_2', 'object_4', 'object_6', 'object_7', 'object_8', 'object_9', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'drift_1', 'drift_2', 'drift_3', 'drift_4', 'drift_5', 'drift_6', 'drift_7', 'drift_8', 'drift_9', 'drift_10', 'drift_11', 'drift_12', 'drift_13', 'drift_14', 'drift_15', 'drift_16', 'drift_17', 'drift_18', 'drift_19', 'drift_20', 'drift_21', 'constant']


Compute beta estimate maps for all 7 object regressors, 1 button press and 1 patch regressor

In [35]:
from nilearn.glm.contrasts import compute_contrast #Cave! This is not for acc. computing contrasts between objects, I am taking the 'raw' effect sizes

# Repeat for all 7 object regressors
object_2_beta_map = first_level_model.compute_contrast('object_2', output_type='effect_size')
object_4_beta_map = first_level_model.compute_contrast('object_4', output_type='effect_size')
object_6_beta_map = first_level_model.compute_contrast('object_6', output_type='effect_size')
object_7_beta_map = first_level_model.compute_contrast('object_7', output_type='effect_size')
object_8_beta_map = first_level_model.compute_contrast('object_8', output_type='effect_size')
object_9_beta_map = first_level_model.compute_contrast('object_9', output_type='effect_size')
object_10_beta_map = first_level_model.compute_contrast('object_10', output_type='effect_size')

# Repeat for button press regressor
button_press_beta_map = first_level_model.compute_contrast('button_press', output_type='effect_size')

# Repeat for 'patch' regressor
#patch_beta_map = first_level_model.compute_contrast('patch', output_type='effect_size') #potentially I forgot this regressor, need to check!

Save beta maps locally for later downstream analyses

In [36]:
import zipfile
import nibabel as nib
from google.colab import files

In [37]:
# Dictionary of beta maps I already created
beta_maps = {
    'object_2': object_2_beta_map,
    'object_4': object_4_beta_map,
    'object_6': object_6_beta_map,
    'object_7': object_7_beta_map,
    'object_8': object_8_beta_map,
    'object_9': object_9_beta_map,
    'object_10': object_10_beta_map,
    'button_press': button_press_beta_map
}

# Save all beta maps
for label, beta_map in beta_maps.items():
    filename = f"sub-04_run3_{label}_beta.nii.gz"
    nib.save(beta_map, filename)

# Zip them
with zipfile.ZipFile("sub-04_run3_betas.zip", "w") as zipf:
    for label in beta_maps.keys():
        zipf.write(f"sub-04_run3_{label}_beta.nii.gz")

# Download the zip
files.download("sub-04_run3_betas.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Concatenated Runs 1 - 3